In [1]:
import os
import torch
import torchvision
import tarfile
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as tt
from torch.utils.data import random_split
from torchvision.utils import make_grid
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data_dir='./data/cifar10'
print(os.listdir(data_dir))
classes=os.listdir(data_dir+"/train")
print(classes)

['test', 'train']
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [3]:
stats=((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010))
train_tfms=tt.Compose([tt.RandomCrop(32,padding=4,padding_mode='reflect'),tt.RandomHorizontalFlip(),tt.ToTensor(),tt.Normalize(*stats)])
valid_tfms=tt.Compose([tt.ToTensor(),tt.Normalize(*stats)])

In [4]:
train_ds=ImageFolder(data_dir+'/train',train_tfms)
valid_ds=ImageFolder(data_dir+'/test',valid_tfms)

In [5]:
batch_size=400
train_dl=DataLoader(train_ds,batch_size,shuffle=True,num_workers=8,pin_memory=True)
valid_dl=DataLoader(valid_ds,batch_size*2,shuffle=False,num_workers=8,pin_memory=True)

In [6]:
def denormalize(images, means, stds):
    means = torch.tensor(means).reshape(1, 3, 1, 1)
    stds = torch.tensor(stds).reshape(1, 3, 1, 1)
    return images * stds + means

In [7]:
def show_batch(dl):
    for images, labels in dl:
        fig, ax = plt.subplots(figsize=(12, 12))
        ax.set_xticks([]); ax.set_yticks([])
        denorm_images = denormalize(images, *stats)
        ax.imshow(make_grid(denorm_images[:64], nrow=8).permute(1, 2, 0).clamp(0,1))
        break

In [31]:
show_batch(train_dl)

In [7]:
class SimpleResidualBlock(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(in_channels=3,out_channels=3,kernel_size=3,stride=1,padding=1)
        self.relu1=nn.ReLU()
        self.conv2=nn.Conv2d(in_channels=3,out_channels=3,kernel_size=3,stride=1,padding=1)
        self.relu2=nn.ReLU()

    def forward(self,x):
        out=self.conv1(x)
        out=self.relu1(out)
        out=self.conv2(out)
        return self.relu2(out+x)

In [15]:
simple_resnet=SimpleResidualBlock()
for images,labels in train_dl:  
    out=simple_resnet(images)
    print(out.shape)
    break

In [14]:
def conv_2d(ni,nf,stride=1,ks=3):
    return nn.Conv2d(in_channels=ni,out_channels=nf,kernel_size=ks,stride=stride,padding=ks//2,bias=False) 

def bn_relu_conv(ni,nf):
    return nn.Sequential(nn.BatchNorm2d(ni),nn.ReLU(inplace=True),conv_2d(ni,nf))

class ResidualBlock(nn.Module):
    def __init__(self,ni,nf,stride=1):
        super().__init__()
        self.bn=nn.BatchNorm2d(ni)
        self.conv1=conv_2d(ni,nf,stride)
        self.conv2=bn_relu_conv(nf,nf)
        self.shortcut=lambda x:x
        if ni!=nf:
            self.shortcut=conv_2d(ni,nf,stride,1)
    
    def forward(self,x):
        x=F.relu(self.bn(x),inplace=True)
        r=self.shortcut(x)
        x=self.conv1(x)
        x=self.conv2(x)*0.2
        return x.add_(r)

In [17]:
def make_group(N,ni,nf,stride):
    start=ResidualBlock(ni,nf,stride)
    rest=[ResidualBlock(nf,nf) for j in range(1,N)]
    return [start]+rest

class Flatten(nn.Module):
    def __init__(self): super().__init__()
    def forward(self,x):return x.view(x.size(0),-1)

class WideResNet(nn.Module):
    def __init__(self,n_groups,N,n_classes,k=1,n_start=16):
        super().__init__()
        layers=[conv_2d(3,n_start)]
        n_channels=[n_start]

        for i in range(n_groups):
            n_channels.append(n_start*(2**i)*k)
            stride=2 if i>0 else 1
            layers+= make_group(N,n_channels[i],n_channels[i+1],stride)

        layers+=[nn.BatchNorm2d(n_channels[3]), nn.ReLU(inplace=True),nn.AdaptiveAvgPool2d(1),Flatten(),nn.Linear(n_channels[3],n_classes)]
        self.features=nn.Sequential(*layers)
    def forward(self,x):return self.features(x)
def wrn_22():
    return WideResNet(n_groups=3,N=3,n_classes=10,k=6)



In [18]:
model=wrn_22()

In [19]:
model

WideResNet(
  (features): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): ResidualBlock(
      (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(16, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Sequential(
        (0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU(inplace=True)
        (2): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (shortcut): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
    )
    (2): ResidualBlock(
      (bn): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Sequential(
        (0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 